In [1]:
import pandas as pd
import itertools


def _float_format(fp: float, max_digits: int = 7) -> str:
    # up to 'max_digits' (point included), except for 0
    # still, take at least the number of digits up to the point
    fp_digits = f'{fp:.{max_digits}f}'
    integer_digits = fp_digits.find('.') - (1 if fp_digits.startswith('-') else 0)
    decimal_digits = max(0, max_digits - integer_digits - 1)
    fp = f'{fp:.{decimal_digits}f}'
    return fp

def table_latex(df, name):
    # convert the dataset to latex, the do some manual postprocessing
    latex = df.to_latex(
        float_format=_float_format,
        caption=f'Change depending on the DataFrame.',
        bold_rows=True,
        escape=False,
        index=False
    )
    latex = latex.replace('\\toprule', '\\hline')
    latex = latex.replace('\\midrule', '\\hline')
    latex = latex.replace('\\bottomrule', '\\hline')
    latex = latex.replace('\\textbf', '\\textit')
    for col in df.columns:
        latex = latex.replace(f'{col} &', '\\textbf{' + col + '} &')
        latex = latex.replace(f'{col} \\', '\\textbf{' + col + '} \\')
    with open(f'./{name}.tex', 'w') as f:
        f.write(latex)

In [2]:
n = 100
path = f"../res/stats/n{n}/"

destroy = ["point", "tour", "neural"]
repair = ["neural"]

## NLNS

In [3]:
nlns = {}
for d in destroy:
    for r in repair:
        key = f"destroy_{d}_repair_{r}"
        nlns[key] = pd.read_csv(path + f"{key}_n{n}.csv", index_col=0)
        nlns[key]["destroy"] = d
        nlns[key]["repair"] = r

## ANLNS

In [4]:
for d1, d2 in itertools.combinations(destroy, 2):
    if d1 != "tour":
        key = f"destroy_{d1}+{d2}_repair_neural"
        nlns[key] = pd.read_csv(path + f"{key}_n{n}.csv", index_col=0)
        nlns[key]["destroy"] = f"{d1}+{d2}"
        nlns[key]["repair"] = "neural"

key = f"destroy_{'+'.join(destroy)}_repair_neural"
nlns[key] = pd.read_csv(path + f"{key}_n{n}.csv", index_col=0)
nlns[key]["destroy"] = '+'.join(destroy)
nlns[key]["repair"] = "neural"

## Baselines

In [5]:
baselines = {"lkh": pd.read_csv(path + f"lkh_n{n}.csv", index_col=0),
             "ortools": pd.read_csv(path + f"ortools_n{n}.csv", index_col=0),
             "scip": pd.read_csv(path + f"scip_n{n}.csv", index_col=0)}

dfs = list(baselines.values())
names = list(baselines.keys())

for df, name in zip(dfs, names):
    df["name"] = name

bl_stats = pd.concat(dfs)

bl_stats = bl_stats.groupby("name").mean()
bl_stats.reset_index(inplace=True)
bl_stats = bl_stats[["name", "cost"]]
bl_stats

,name,cost
0,lkh,15.749453
1,ortools,16.359535
2,scip,25.840413


## Generate LaTex Table

In [6]:
dfs = list(nlns.values())
stats = pd.concat(dfs)

mean_stats = stats.groupby(["destroy", "repair"]).mean()
mean_stats.reset_index(inplace=True)
mean_stats = mean_stats[["destroy", "repair", "cost"]]
mean_stats

,destroy,repair,cost
0,neural,neural,16.035981
1,point,neural,15.969997
2,point+neural,neural,15.933830
3,point+tour,neural,16.031496
4,point+tour+neural,neural,15.936926
5,tour,neural,16.112183


In [7]:
# table_latex(mean_stats, "mean_stats")